# Libraries 

In [ ]:
# trio!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Import Train/Test Set

This time we use sampling to reduce training time.

In [ ]:
df_train = pd.read_csv('train.csv').sample(10000, random_state=2021)
df_test = pd.read_csv('test.csv').sample(3000, random_state=2021)

In [ ]:
features_list = ['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'i_d', 'i_q']
target = 'stator_yoke'

In [ ]:
X_train = df_train[features_list].to_numpy()
y_train = df_train[target].to_numpy()

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test = df_test[features_list].to_numpy()
y_test = df_test[target].to_numpy()

In [ ]:
X_test.shape, y_test.shape

# Data Preprocessing

In [ ]:
X_scaler = MinMaxScaler()

In [ ]:
X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#  Predictive Models

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def plot_test(reg, X, y):
    y_pred = reg.predict(X)
    plt.plot(y, alpha=0.5)
    plt.plot(y_pred, alpha=0.5)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    print(f'RMSE: {rmse}')

## Ridge 

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [ ]:
param_grid_ridge = {
    "alpha": [0.01, 0.2, 0.4, 0.6, 0.8, 1],
    "solver": ["cholesky", "saga"]
}

In [ ]:
grid_search_ridge = GridSearchCV(ridge, param_grid_ridge, cv=5)

In [ ]:
grid_search_ridge.fit(X_train_scaled, y_train)

In [ ]:
grid_search_ridge.best_params_

In [ ]:
grid_search_ridge.cv_results_

In [ ]:
df_grid_search_ridge = pd.DataFrame(grid_search_ridge.cv_results_)
df_grid_search_ridge.head(5)

In [ ]:
df_grid_search_ridge.sort_values(["param_solver", "param_alpha", "mean_test_score"], inplace=True)

In [ ]:
sns.lineplot(x="param_alpha", y="mean_test_score", hue="param_solver", data=df_grid_search_ridge)

In [ ]:
grid_search_ridge.score(X_test_scaled, y_test)

In [ ]:
plot_test(grid_search_ridge, X_test_scaled, y_test)

## Lasso

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso()

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [ ]:
param_grid_lasso = {
    "alpha": [0.01, 0.2, 0.4, 0.6, 0.8, 1]
}

In [ ]:
grid_search_lasso = GridSearchCV(lasso, param_grid=param_grid_lasso, cv=5)

In [ ]:
grid_search_lasso.fit(X_train_scaled, y_train)

In [ ]:
grid_search_lasso.best_params_

In [ ]:
grid_search_lasso.cv_results_

In [ ]:
df_grid_search_lasso = pd.DataFrame(grid_search_lasso.cv_results_)
df_grid_search_lasso.head(5)

In [ ]:
df_grid_search_lasso.sort_values(["param_alpha", "mean_test_score"], inplace=True)

In [ ]:
sns.lineplot(x="param_alpha", y="mean_test_score", data=df_grid_search_lasso)

In [ ]:
grid_search_lasso.score(X_test_scaled, y_test)

In [ ]:
plot_test(grid_search_lasso, X_test_scaled, y_test)

## Decision Tree Regressor 

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor()

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

In [ ]:
param_grid_dt_reg = {
    "max_depth": [5, 7, 10, 15, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 10, 20],
}

In [ ]:
grid_search_dt_reg = GridSearchCV(dt_reg, param_grid_dt_reg, cv=5)

In [ ]:
grid_search_dt_reg.fit(X_train, y_train)

In [ ]:
grid_search_dt_reg.best_params_

In [ ]:
grid_search_dt_reg.cv_results_

In [ ]:
df_grid_search_dt_reg = pd.DataFrame(grid_search_dt_reg.cv_results_)
df_grid_search_dt_reg.head(5)

In [ ]:
df_grid_search_dt_reg.sort_values(["param_max_depth", "param_min_samples_leaf", "param_min_samples_split", "mean_test_score"], inplace=True)

In [ ]:
g = sns.FacetGrid(df_grid_search_dt_reg, col="param_min_samples_split")
g.map(sns.lineplot, "param_max_depth", "mean_test_score", "param_min_samples_leaf");

In [ ]:
grid_search_dt_reg.score(X_test, y_test)

In [ ]:
plot_test(grid_search_dt_reg, X_test, y_test)

## Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor()

In [ ]:
param_grid_rf_reg = {
    "max_depth": [5, 7, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 10, 20],
}

In [ ]:
grid_search_rf_reg = GridSearchCV(rf_reg, param_grid_rf_reg, cv=5)

In [ ]:
grid_search_rf_reg.fit(X_train, y_train)

In [ ]:
grid_search_rf_reg.best_params_

In [ ]:
grid_search_rf_reg.cv_results_

In [ ]:
df_grid_search_rf_reg = pd.DataFrame(grid_search_rf_reg.cv_results_)
df_grid_search_rf_reg.head(5)

In [ ]:
df_grid_search_rf_reg.sort_values(["param_max_depth", "param_min_samples_leaf", "param_min_samples_split", "mean_test_score"], inplace=True)

In [ ]:
g = sns.FacetGrid(df_grid_search_rf_reg, col="param_min_samples_split")
g.map(sns.lineplot, "param_max_depth", "mean_test_score", "param_min_samples_leaf");

In [ ]:
grid_search_rf_reg.score(X_test, y_test)

In [ ]:
plot_test(grid_search_rf_reg, X_test, y_test)

## AdaBoost Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor()

In [ ]:
param_grid_rf_reg = {
    "max_depth": [5, 7, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 10, 20],
}

In [ ]:
grid_search_rf_reg = GridSearchCV(rf_reg, param_grid_rf_reg, cv=5)

In [ ]:
grid_search_rf_reg.fit(X_train, y_train)

In [ ]:
grid_search_rf_reg.best_params_

In [ ]:
grid_search_rf_reg.cv_results_

In [ ]:
df_grid_search_rf_reg = pd.DataFrame(grid_search_rf_reg.cv_results_)
df_grid_search_rf_reg.head(5)

In [ ]:
df_grid_search_rf_reg.sort_values(["param_max_depth", "param_min_samples_leaf", "param_min_samples_split", "mean_test_score"], inplace=True)

In [ ]:
g = sns.FacetGrid(df_grid_search_rf_reg, col="param_min_samples_split")
g.map(sns.lineplot, "param_max_depth", "mean_test_score", "param_min_samples_leaf");

In [ ]:
grid_search_rf_reg.score(X_test, y_test)

In [ ]:
plot_test(grid_search_rf_reg, X_test, y_test)

## Gradient Boosting Regressor 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor()

In [ ]:
param_grid_rf_reg = {
    "max_depth": [5, 7, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 10, 20],
}

In [ ]:
grid_search_rf_reg = GridSearchCV(rf_reg, param_grid_rf_reg, cv=5)

In [ ]:
grid_search_rf_reg.fit(X_train, y_train)

In [ ]:
grid_search_rf_reg.best_params_

In [ ]:
grid_search_rf_reg.cv_results_

In [ ]:
df_grid_search_rf_reg = pd.DataFrame(grid_search_rf_reg.cv_results_)
df_grid_search_rf_reg.head(5)

In [ ]:
df_grid_search_rf_reg.sort_values(["param_max_depth", "param_min_samples_leaf", "param_min_samples_split", "mean_test_score"], inplace=True)

In [ ]:
g = sns.FacetGrid(df_grid_search_rf_reg, col="param_min_samples_split")
g.map(sns.lineplot, "param_max_depth", "mean_test_score", "param_min_samples_leaf");

In [ ]:
grid_search_rf_reg.score(X_test, y_test)

In [ ]:
plot_test(grid_search_rf_reg, X_test, y_test)

## XGBoost Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor()

In [ ]:
param_grid_rf_reg = {
    "max_depth": [5, 7, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 10, 20],
}

In [ ]:
grid_search_rf_reg = GridSearchCV(rf_reg, param_grid_rf_reg, cv=5)

In [ ]:
grid_search_rf_reg.fit(X_train, y_train)

In [ ]:
grid_search_rf_reg.best_params_

In [ ]:
grid_search_rf_reg.cv_results_

In [ ]:
df_grid_search_rf_reg = pd.DataFrame(grid_search_rf_reg.cv_results_)
df_grid_search_rf_reg.head(5)

In [ ]:
df_grid_search_rf_reg.sort_values(["param_max_depth", "param_min_samples_leaf", "param_min_samples_split", "mean_test_score"], inplace=True)

In [ ]:
g = sns.FacetGrid(df_grid_search_rf_reg, col="param_min_samples_split")
g.map(sns.lineplot, "param_max_depth", "mean_test_score", "param_min_samples_leaf");

In [ ]:
grid_search_rf_reg.score(X_test, y_test)

In [ ]:
plot_test(grid_search_rf_reg, X_test, y_test)